# OceanCast
## 1. 模型简介
该项目实现海浪(海流)细时空分辨率预测，模型参照 [FourCastNet：使用自适应傅里叶神经算子的全球数据驱动高分辨率天气模型](https://arxiv.org/abs/2202.11214) 的代码构建。<br>

**模型结构**<br>
主要参考FourCastNet中基于傅里叶算子的AFNO网络结构

**模型简介**<br>
OceanCast 是一种全球数据驱动的海浪/海流/洋流预报模型，可提供1° 分辨率的精确超短期全球预测。
OceanCast 可准确预测高分辨率、快速时间尺度变量，例如海浪高度、海浪周期、海浪方向(海表温、海表盐、海面高度或洋流)等。
通过修改conf/oceancast.yaml中待预测的input/output_type，可切换海浪/海流/洋流预报模型

## 2. 软件环境准备

### 2.1 基于dtk适配软件
<div class="alert alert-warning"> WARNING：镜像中环境已配置，此步骤省略 </div>

### 2.2 软件环境检查


In [4]:
# 检查torch版本
import torch
import os
import onescience

version = torch.__version__
num = float(version[:3])
# assert num == 1.10

# 检查硬件环境
device = "cpu"
if os.system('rocm-smi 2>/dev/null || hy-smi 2>/dev/null')==0:
    device = "dtk"

elif os.system('nvidia-smi 2>/dev/null')==0: 
    device = "cuda"

print("torch version:", version)
print("device =", device)
print("onescience version:", onescience.__version__)


============================ System Management Interface =============================
DCU     Temp     AvgPwr     Perf     PwrCap     VRAM%      DCU%      Mode     
0       50.0C    24.0W      auto     300.0W     0%         0%        N/A      
1       49.0C    25.0W      auto     300.0W     0%         0%        N/A      
2       50.0C    24.0W      auto     300.0W     0%         0%        N/A      
3       50.0C    27.0W      auto     300.0W     0%         0%        N/A      
=================================== End of SMI Log ===================================

torch version: 2.3.0
device = dtk
onescience version: 0.1.0


### 2.2 软件依赖安装


<div class="alert alert-warning"> WARNING: 镜像中环境已配置，此步骤省略 </div>

<div class="alert alert-note" style="color: blue;">
Note: 存在部分文件更新，请卸载当前环境下的 onescience 包，并安装当前目录的 whl 包。
    
请在最顶层目录(包含setup.py文件的目录)，通过命令行，执行下述安装指令：
</div>

In [ ]:
# !pip install -e .
!pwd

## 3. 素材准备
### 3.1 数据集准备

以海浪预报为例，数据集目录组织方式如下(海流预报中前3个分别对应Ocean_SST, Ocean_SSS, Ocean_SSH)：

数据集存放于/public/SothisAI/sharingCenter/data/easyscience/Ocean # 该路径包含所有数据集

风速数据首先需要按照3.2计算为正弦、余弦、强度，并存放在本目录下，其余的海浪(海流)数据则存放于原始目录中。

训练-验证-测试数据集按照8：1：1组织数据

共26年数据，分为20-3-3

```
oceancast
│   README.md
│
└───conf
│   │   oceancast.yaml(配置文件)
│
└───data(共26年数据，按照8: 1: 1分割，因此训练：验证：测试=20: 3: 3)
└──────Wind_Sin(风向正弦值，根据原始U_V计算得到角度)
│   │   1995.h5
│   │   ...
│   │   2020.h5
└──────Wind_Cos(风向正弦值，根据原始U_V计算得到角度)
│   │   1995.h5
│   │   ...
│   │   2020.h5
└──────Wind_Strength(风速大小)
│   │   1995.h5
│   │   ...
│   │   2020.h5
│
└───means_stds(各变量的均值和标准差，用于归一化)
│   │   Ocean_SST_means.npy
│   │   ...
│   │   Wind_Strength_stds.npy
│
└───WaveMask.pt(海洋-陆地掩码)
│
└───data_preprocess.py(计算风速相关数据)
│
└─────────────────────

均值和方差需要根据get_means_stds.py统一计算，目前已存放于means_stds，可跳过计算

```

### 3.2 新数据计算

以海浪预报为例，需要按照data_preprocess.py计算风速的正、余弦值以及风速强度

海流预报也需计算

```

In [ ]:
import os
# 设置环境变量待修改
!python3 data_preprocess.py 

## 4 训练


### 4.1 单卡训练
运行`python3 train_oceancast.py`进行训练

主要参数介绍，具体参数可在conf/oceancast.yaml中修改：
- name：模型名称，训练阶段名称为`Oceancast`
- model_path：模型文件及loss输出保存路径，默认`checkpoint/weight`
- maskpath: 陆地掩码，默认`./WaveMask.pt`
- patience: 早停轮数，默认50
- max_epochs：最大训练步数，因为有早停，因此默认设为10000
- batch_size: 一次性训练样本数量，默认16，如果溢出可根据DCU显存调整

In [ ]:
import os
# 设置环境变量待修改
!python3 train_oceancast.py 

### 4.2 多卡训练
运行`torchrun --nproc_per_node=4 --nnodes=1 --rdzv_id=1000 --rdzv_backend=c10d --max_restarts=0 --master_addr="localhost" --master_port=29500 train_oceancast.py`进行训练

主要参数介绍，其他参数参见单卡介绍：
- nproc_per_node=4：卡的数量，默认4卡
- nnodes=1：节点数量，默认单节点4卡

In [ ]:
import os
import torch
import sys

nproc_per_node = torch.cuda.device_count()
print(f'This machine has {nproc_per_node} DCU/GPU devices')

if nproc_per_node > 1:
# 设置环境变量待修改
    !torchrun \
    --nproc_per_node=4 \
    --nnodes=1 \
    --rdzv_id=1000 \
    --rdzv_backend=c10d \
    --max_restarts=0 \
    --master_addr="localhost" \
    --master_port=29500 \
    train_oceancast.py
else:
    print('Only one DCU/GPU, please use single-device to train.')

### 4.3 Pipeline模式

该模式将AFNO切分为模型并行模式(Pipeline parallel, pp), 用于提供dualpipe加速的benchmark

在实际情况下, 几乎所有数据并行(Data Distributed Parallel, ddp)模式都比pp模式要快

此处仅作为案例展示，当模型大到必须用pp模式时，dualpipe具备应用价值。

In [ ]:
#!/bin/bash

!export HIP_VISIBLE_DEVICES=0,1,2,3 
!export MASTER_ADDR="localhost"
!export MASTER_PORT="29500"
!export WORLD_SIZE=4
!export NCCL_DEBUG=ERROR

!torchrun --nproc_per_node=4  train_oceancast_pipeline.py

### 4.4 Dualpipe加速的Pipeline模式

基于4.3构建的pp模式，使用deepseek提供的dualpipeV技术进行加速

In [ ]:
#!/bin/bash

!export HIP_VISIBLE_DEVICES=0,1,2,3 
!export MASTER_ADDR="localhost"
!export MASTER_PORT="29500"
!export WORLD_SIZE=4
!export NCCL_DEBUG=ERROR

!torchrun --nproc_per_node=4  train_oceancast_dualpipe.py

## 5. 许可证
Oceancast 项目（包括代码和模型参数）在[Apache 2.0](https://github.com/bytedance/Protenix/blob/main/LICENSE)许可下提供，可免费用于学术研究和商业用途。
